###**Q1**.  Use Lucidchart to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [1]:
import sqlite3
import pandas as pd
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# 2.2 Create the database and save it in a file called store.db, and create the four tables as described above.
# Your code here . . .
#PRAGMA foreign_keys=ON;
gdrive='drive/MyDrive/CS 133/week10_sql/'
Path(gdrive+'store.db').touch()
conn = sqlite3.connect(gdrive+'store.db') # creates a connection to the database
c = conn.cursor() # the cursor is what we'll actually use

SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS product (
             product_id INTEGER PRIMARY KEY,
             name TEXT NOT NULL,
             price REAL
             )'''
c.execute(SQL_CreateTable)

SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS customer (
             customer_id INTEGER PRIMARY KEY,
             name TEXT NOT NULL,
             email TEXT NOT NULL
             )'''
c.execute(SQL_CreateTable)

SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS purchase_order (
            order_id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            date TEXT NOT NULL,
            FOREIGN KEY (customer_id)
              REFERENCES customer (customer_id)
                ON DELETE CASCADE
                ON UPDATE NO ACTION
            )'''
c.execute(SQL_CreateTable)

SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS order_item (
            order_id INTEGER,
            product_id INTEGER,
            quantity INTEGER,
            FOREIGN KEY (order_id)
              REFERENCES purchase_order (order_id)
                ON DELETE CASCADE
                ON UPDATE NO ACTION,
            FOREIGN KEY (product_id)
              REFERENCES product (product_id)
                ON DELETE CASCADE
                ON UPDATE NO ACTION
            )'''
c.execute(SQL_CreateTable)

In [3]:
# 2.3 Load the data in the csv files into the corresponding table.
# Your code here . . .
product = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/product.csv")
customer = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv")
order_item = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv")
purchase_order = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv")

In [4]:
# 2.4 Commit so that the data loaded to the tables to officially written to the tables.
# Your code here . . .
product.to_sql('product', conn, if_exists='append', index=False)
customer.to_sql('customer', conn, if_exists='append', index=False)
order_item.to_sql('order_item', conn, if_exists='append', index=False)
purchase_order.to_sql('purchase_order', conn, if_exists='append', index=False)

3

In [5]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
# Your code here . . .
c.execute("SELECT * FROM product")
results = c.fetchall()
print(results)
c.execute("SELECT * FROM customer")
results = c.fetchall()
print(results)
c.execute("SELECT * FROM order_item")
results = c.fetchall()
print(results)
c.execute("SELECT * FROM purchase_order")
results = c.fetchall()
print(results)

[(0, 'bicycle', 400.0), (1, 'helmet', 45.0), (2, 'gloves', 23.0), (3, 'chain', 48.0)]
[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu')]
[(0, 0, 1), (0, 1, 1), (1, 2, 2), (1, 3, 1)]
[(0, 0, '2020-10-19'), (1, 0, '2020-10-20'), (2, 1, '2020-10-20')]


In [12]:
# 2.6 Execute a query using SELECT statement that queries with JOIN tables to find the purchase date,
#    the products and quantities that a particular customer has purchased.
# Your code here . . .
SQL_JointQuery = '''SELECT purchase_order.date, product.name,
                order_item.quantity FROM product
                JOIN order_item using (product_id)
                JOIN purchase_order using (order_id)
                JOIN customer using (customer_id)
                WHERE customer.name='Wendy Lee'
                '''
c.execute(SQL_JointQuery)
joint_results = c.fetchall()
print(joint_results)

[('2020-10-19', 'bicycle', 1), ('2020-10-19', 'helmet', 1), ('2020-10-20', 'gloves', 2), ('2020-10-20', 'chain', 1)]
